In [10]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [11]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import emeval.viz.geojson as ezgj
import pandas as pd

In [12]:
# Metrics helpers
import emeval.metrics.dist_calculations as emd
import emeval.metrics.reference_trajectory as emr

In [13]:
# For computation
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt

In [14]:
import geopandas as gpd
import shapely as shp
import folium

In [15]:
DATASTORE_LOC = "bin/data"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

After iterating over 1 entries, entry found
Found spec = Round trip car and bike trip in the South Bay
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
After iterating over 1 entries, entry found
Found spec = Multi-modal car scooter BREX trip to San Jose
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
After iterating over 1 entries, entry found
Found spec = Multimodal multi-train, multi-bus, ebike trip to UC Berkeley
Evaluation ran from 2019-07-16T00:00:00-07:00 -> 2020-04-30T00:00:00-07:00


In [16]:
import importlib
importlib.reload(emr)

<module 'emeval.metrics.reference_trajectory' from '/Users/kinjal/Code/openpath/mobilitynet-analysis-scripts/emeval/metrics/reference_trajectory.py'>

In [17]:
pv_la = eipv.PhoneView(sd_la)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 86 total -> 0 calibration

In [18]:
pv_sj = eipv.PhoneView(sd_sj)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 86 total -> 0 calibration

In [19]:
pv_ucb = eipv.PhoneView(sd_ucb)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 102 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 90 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 90 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 90 total -> 0 calibratio

## Spatial-temporal error calculation

In [20]:
def get_reference_trajectory_input_tree(pv):
    ref_tree = {}
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                if r["eval_role_base"] != "accuracy_control":
                    continue
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        # print(f"Retrieving ground truth for range {arrow.get(sr['start_ts'])} -> {arrow.get(sr['end_ts'])}")
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"], sr["start_ts"], sr["end_ts"])
                        section_gt_shapes = gpd.GeoSeries(eisd.SpecDetails.get_shapes_for_leg(section_gt_leg))
                        if len(section_gt_shapes) == 1:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) == 0:
                            print("No sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue
                            
                        print("Processing travel leg %s, %s, %s, %s, %s" %
                              (phone_os, phone_label, r["eval_role_base"], tr["trip_id_base"], sr["trip_id_base"]))
                        # print(f"Ground truth for trip range {arrow.get(tr['start_ts'])} and {arrow.get(tr['end_ts'])} is "
                        #       + f"START: {section_gt_leg['start_loc']['properties']['valid_start_fmt_date']} -> {section_gt_leg['start_loc']['properties']['valid_end_fmt_date']} "
                        #       + f"END:  {section_gt_leg['end_loc']['properties']['valid_start_fmt_date']} -> {section_gt_leg['end_loc']['properties']['valid_end_fmt_date']} ")
                        sec_name = tr["trip_id_base"] + "/" + sr["trip_id_base"] + "_" + str(r_idx)
                        if sec_name not in ref_tree:
                            ref_tree[sec_name] = {
                                "trip_id": tr["trip_id_base"],
                                "section_id": sr["trip_id_base"],
                                "run": r_idx,
                                "ground_truth": {
                                    "leg": section_gt_leg
                                }
                            }
                        
                        assert sec_name in ref_tree
                        e = ref_tree[sec_name]
                        # This is a GeoDataFrame
                        # section_measured_points = get_travel_trajectory(sr['location_df'], polygons)
                        section_measured_points = sr["location_df"]
                        if "temporal_control" not in e:
                            e["temporal_control"] = {}
                            e["start_ts"] = sr["start_ts"]
                            e["end_ts"] = sr["end_ts"]
                        e["temporal_control"][phone_os] = sr
    return ref_tree

def fill_ref_tree_entry(pv, e, function, include_ends=False):
    print("Considering entry %s %s %s and include_ends=%s" % (e["trip_id"], e["section_id"], e["run"], include_ends))
    curr_tz = pv.spec_details.eval_tz
    assert "android" in e["temporal_control"] and "ios" in e["temporal_control"]
    out = emr.ref_and_stats(e, function, 25, tz=curr_tz, include_ends=include_ends)
    if out is not None:
        e["reference_df"], e["reference_stats"] = out

def get_reference_trajectory_tree(pv, ref_tree_root, function, include_ends=False):
    curr_ref_tree = get_reference_trajectory_input_tree(pv)
    ref_tree_root[pv.spec_details.CURR_SPEC_ID] = curr_ref_tree
    [fill_ref_tree_entry(pv, e, function, include_ends) for e in curr_ref_tree.values()]

In [21]:
import arrow

In [22]:
# %%capture --no-stdout
# ref_tree_no_ends = {}
# get_reference_trajectory_tree(pv_la, ref_tree_no_ends)
# get_reference_trajectory_tree(pv_sj, ref_tree_no_ends)
# get_reference_trajectory_tree(pv_ucb, ref_tree_no_ends)

In [23]:
import importlib

In [24]:
importlib.reload(emr)

<module 'emeval.metrics.reference_trajectory' from '/Users/kinjal/Code/openpath/mobilitynet-analysis-scripts/emeval/metrics/reference_trajectory.py'>

In [25]:
# breaking_e = ref_tree['train_bus_ebike_mtv_ucb']['mtv_to_berkeley_sf_bart/walk_to_bus_1']

In [26]:
# %%capture --no-stdout
# ref_trajectory = emr.final_ref_ensemble(breaking_e, 25, tz="America/Los_angeles", include_ends=True)

In [27]:
# ref_trajectory[1].tail(n=20)

In [28]:
# curr_map = folium.Map()
# gt_leg_gj = pv_ucb.spec_details.get_geojson_for_leg(breaking_e["ground_truth"]["leg"])
# sensed_section_gj = ezgj.get_geojson_for_loc_df(breaking_e["temporal_control"]["ios"]["location_df"].tail(n=15))
# gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
# sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
# ref_trajectory_gj = ezgj.get_geojson_for_loc_df(ref_trajectory[1], color="blue")
# ref_leg_gj_feature = folium.GeoJson(ref_trajectory_gj, name="reference_values")
# curr_map.add_child(gt_leg_gj_feature)
# curr_map.add_child(sensed_leg_gj_feature)
# curr_map.add_child(ref_leg_gj_feature)
# curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
# folium.LayerControl().add_to(curr_map)
# curr_map

In [29]:
# map_list = []
# # for i in range(12):
# for i in range(0):
#     breaking_e = ref_tree['train_bus_ebike_mtv_ucb']['mtv_to_berkeley_sf_bart/walk_to_bus_%d' % i]
#     curr_map = folium.Map()
#     gt_leg_gj = pv_ucb.spec_details.get_geojson_for_leg(breaking_e["ground_truth"]["leg"])
#     sensed_section_gj = ezgj.get_geojson_for_loc_df(breaking_e["temporal_control"]["android"]["location_df"])
#     gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
#     sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
#     curr_map.add_child(gt_leg_gj_feature)
#     curr_map.add_child(sensed_leg_gj_feature)
#     curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
#     folium.LayerControl().add_to(curr_map)
#     map_list.append(curr_map)

In [30]:
# import branca.element as bre
# fig = bre.Figure()
# for i, curr_map in enumerate(map_list):
#     fig.add_subplot(4, 3, i+1).add_child(curr_map)
# fig

In [31]:
# map_list = []
# # for phone_os in ["android", "ios"]:
# for phone_os in []:
#     breaking_e = ref_tree['train_bus_ebike_mtv_ucb']['mtv_to_berkeley_sf_bart/walk_to_bus_1']
#     curr_map = folium.Map()
#     gt_leg_gj = pv_ucb.spec_details.get_geojson_for_leg(breaking_e["ground_truth"]["leg"])
#     sensed_section_gj = ezgj.get_geojson_for_loc_df(breaking_e["temporal_control"][phone_os]["location_df"])
#     gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
#     sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
#     section_gt_shapes = breaking_e["ground_truth"]["gt_shapes"]
#     filtered_loc = emd.filter_geo_df(breaking_e["temporal_control"][phone_os]["location_df"],
#                       section_gt_shapes.filter(["start_loc","end_loc"]))
#     filtered_section_gj = ezgj.get_geojson_for_loc_df(filtered_loc)
#     filtered_section_gj_feature = folium.GeoJson(filtered_section_gj, name="filtered_values")
#     curr_map.add_child(gt_leg_gj_feature)
#     curr_map.add_child(sensed_leg_gj_feature)
#     curr_map.add_child(filtered_section_gj_feature)
#     curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
#     folium.LayerControl().add_to(curr_map)
#     map_list.append(curr_map)

In [32]:
# import branca.element as bre
# fig = bre.Figure()
# for i, curr_map in enumerate(map_list):
#     fig.add_subplot(1, 2, i+1).add_child(curr_map)
# fig

In [33]:
# map_list = []
# # for phone_os in ["android", "ios"]:
# for phone_os in []:
#     breaking_e = ref_tree['train_bus_ebike_mtv_ucb']['mtv_to_berkeley_sf_bart/walk_to_bus_11']
#     curr_map = folium.Map()
#     gt_leg_gj = pv_ucb.spec_details.get_geojson_for_leg(breaking_e["ground_truth"]["leg"])
#     sensed_section_gj = ezgj.get_geojson_for_loc_df(breaking_e["temporal_control"][phone_os]["location_df"])
#     gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
#     sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
#     section_gt_shapes = breaking_e["ground_truth"]["gt_shapes"]
#     filtered_loc = emd.filter_geo_df(breaking_e["temporal_control"][phone_os]["location_df"],
#                       section_gt_shapes.filter(["start_loc","end_loc"]))
#     filtered_section_gj = ezgj.get_geojson_for_loc_df(filtered_loc)
#     filtered_section_gj_feature = folium.GeoJson(filtered_section_gj, name="filtered_values")
#     curr_map.add_child(gt_leg_gj_feature)
#     curr_map.add_child(sensed_leg_gj_feature)
#     curr_map.add_child(filtered_section_gj_feature)
#     curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
#     folium.LayerControl().add_to(curr_map)
#     map_list.append(curr_map)

In [34]:
# import branca.element as bre
# fig = bre.Figure()
# for i, curr_map in enumerate(map_list):
#     fig.add_subplot(1, 2, i+1).add_child(curr_map)
# fig

In [35]:
import os

In [39]:
def save_ref_tree(ref_tree, function, include_ends=False):
    for timeline in ref_tree.keys():
        print("Working with timeline %s" % timeline)
        tl = ref_tree[timeline]
        prefix = "with_ends" if include_ends else "no_ends"
        for trip_and_leg_and_run in tl.keys():
            curr_run = tl[trip_and_leg_and_run]
            fn = "bin/ref_trees/" + function + "/" + timeline+"/"+prefix+"/"+trip_and_leg_and_run
            save_dir = os.path.dirname(fn)
            os.makedirs(save_dir, exist_ok=True)
            print("Saving data to file %s" % fn)
            if "reference_df" not in curr_run:
                print("Found missing 'reference_df', skipping")
            else:
                curr_run.get('reference_df', pd.DataFrame()).to_csv(fn)

In [37]:
# save_ref_tree(ref_tree_no_ends, include_ends=False)

In [40]:
%%capture --no-stdout
functions = ['tf', 'ct', 'dtw']
ref_trees_all = []
for function in functions:
    ref_tree_with_ends = {}
    get_reference_trajectory_tree(pv_la, ref_tree_with_ends, function, include_ends=True)
    get_reference_trajectory_tree(pv_sj, ref_tree_with_ends, function, include_ends=True)
    get_reference_trajectory_tree(pv_ucb, ref_tree_with_ends, function, include_ends=True)
    ref_trees_all.append(ref_tree_with_ends)
    save_ref_tree(ref_tree_with_ends, function, include_ends=True)

No ground truth route for suburb_city_driving_weekend walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_city_driving_weekend, suburb_city_driving_weekend
No ground truth route for suburb_city_driving_weekend walk_end, must be polygon, skipping...
No ground truth route for suburb_bicycling walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_bicycling, suburb_bicycling
No ground truth route for suburb_bicycling walk_end, must be polygon, skipping...
No ground truth route for suburb_city_driving_weekend walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_city_driving_weekend, suburb_city_driving_weekend
No ground truth route for suburb_city_driving_weekend walk_end, must be polygon, skipping...
No ground truth route for suburb_bicycling walk_start, must be polygon, skipping...
Processing travel leg

In [48]:
with open("segment_stats_all.txt", "w") as f:
    tree = ref_trees_all[0]
    for trip in tree.keys():
        f.write("trip: %s\n" % trip)
        for segment in tree[trip].keys():
            f.write("\tsegment: %s\n" % segment)
            segment_stats = {}
            if "reference_stats" in ref_trees_all[0][trip][segment]:
                segment_stats["tf"] = ref_trees_all[0][trip][segment]["reference_stats"]
                f.write("\t\ttf:%s\n" % segment_stats["tf"])
            if "reference_stats" in ref_trees_all[1][trip][segment]:
                segment_stats["ct"] = ref_trees_all[1][trip][segment]["reference_stats"]
                f.write("\t\tct:%s\n" % segment_stats["ct"])
            if "reference_stats" in ref_trees_all[2][trip][segment]:
                segment_stats["dtw"] = ref_trees_all[2][trip][segment]["reference_stats"]
                f.write("\t\tdtw:%s\n" % segment_stats["dtw"])
            # f.write("trip: %s\tsegment: %s\t%s\t%s\t%s\n" % (trip, segment, segment_stats["tf"], segment_stats["ct"], segment_stats["dtw"]))
            f.write("\n")

In [ ]:
# importlib.reload(emr)

In [ ]:
# emr.final_ref_ensemble(broken_e, include_ends=True)

In [ ]:
save_ref_tree(ref_tree_with_ends, include_ends=True)

Working with timeline unimodal_trip_car_bike_mtv_la
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_city_driving_weekend/suburb_city_driving_weekend_0
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_bicycling/suburb_bicycling_0
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_city_driving_weekend/suburb_city_driving_weekend_1
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_bicycling/suburb_bicycling_1
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_city_driving_weekend/suburb_city_driving_weekend_2
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_bicycling/suburb_bicycling_2
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_city_driving_weekend/suburb_city_driving_weekend_3
Saving data to file bin/data/unimodal_trip_car_bike_mtv_la/with_ends/suburb_bicycling/suburb_bicycling_3
Saving data to file 

In [ ]:
ref_tree = ref_tree_with_ends
(len(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/walk_to_bus_11"]['temporal_control']['android']['location_df']), len(ref_tree_with_ends["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/walk_to_bus_11"]['temporal_control']['ios']['location_df']))

(293, 93)

In [ ]:
def get_spatio_temporal_errors(pv):
    spatial_error_df = pd.DataFrame()
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                run_errors = []
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    trip_errors = []
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"], sr["start_ts"], sr["end_ts"])
                        section_gt_shapes = gpd.GeoSeries(eisd.SpecDetails.get_shapes_for_leg(section_gt_leg))
                        if len(section_gt_shapes) == 1:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) < 2:
                            print("Too few sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue
                            
                        print("Processing travel leg %s, %s, %s, %s, %s" %
                              (phone_os, phone_label, r["eval_role_base"], tr["trip_id_base"], sr["trip_id_base"]))

                        # This is a GeoDataFrame
                        section_geo_df = emd.to_geo_df(sr["location_df"])
                        filtered_section_geo_df = emd.filter_geo_df(section_geo_df, section_gt_shapes)
                        if len(filtered_section_geo_df) < 2:
                            print("Too few filtered locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue

                        
                        # This is a GeoDataFrame
                        eval_df = emr.get_int_aligned_trajectory(filtered_section_geo_df, tz=pv.spec_details.eval_tz)
                        sr["resampled_df"] = eval_df
                        spec_name = tr["trip_id_base"]+"/"+sr["trip_id_base"]+"_"+str(r_idx)
                        reference_df = ref_tree[pv.spec_details.CURR_SPEC_ID][spec_name]["reference_df"]

                        # Match these up by timestamp
                        merged_df = pd.merge(eval_df, reference_df, on="ts", how="inner", suffixes=("_e", "_r")).sort_values(by="ts", axis="index")
                        print("len(eval_df) = %d, len(reference_df) = %d len(merged_df) = %d" % (len(eval_df), len(reference_df), len(merged_df)))
                        merged_df["t_distance"] = emd.to_utm_series(gpd.GeoSeries(merged_df.geometry_e)).distance(emd.to_utm_series(gpd.GeoSeries(merged_df.geometry_r)))
                        ne = len(merged_df)
                        curr_spatial_error_df = gpd.GeoDataFrame({"error": merged_df["t_distance"],
                            "ts": merged_df.ts, "geometry": merged_df.geometry_e,
                             "phone_os": np.repeat(phone_os, ne),
                             "phone_label": np.repeat(phone_label, ne),
                             "role": np.repeat(r["eval_role_base"], ne),
                             "timeline": np.repeat(pv.spec_details.CURR_SPEC_ID, ne), 
                              "run": np.repeat(r_idx, ne),
                              "trip_id": np.repeat(tr["trip_id_base"], ne),
                              "section_id": np.repeat(sr["trip_id_base"], ne)
                        })
                        spatial_error_df = pd.concat([spatial_error_df, curr_spatial_error_df], axis="index")
    return spatial_error_df

In [ ]:
st_errors_df = pd.DataFrame()
st_errors_df = pd.concat([st_errors_df, get_spatio_temporal_errors(pv_la)], axis="index")
st_errors_df = pd.concat([st_errors_df, get_spatio_temporal_errors(pv_sj)], axis="index")
st_errors_df = pd.concat([st_errors_df, get_spatio_temporal_errors(pv_ucb)], axis="index")

No ground truth route for suburb_city_driving_weekend walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_city_driving_weekend, suburb_city_driving_weekend
After filtering against polygons 3, 259 -> 180
len(eval_df) = 666, len(reference_df) = 578 len(merged_df) = 535
No ground truth route for suburb_city_driving_weekend walk_end, must be polygon, skipping...
No ground truth route for suburb_bicycling walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_bicycling, suburb_bicycling
After filtering against polygons 3, 1163 -> 935
len(eval_df) = 954, len(reference_df) = 1146 len(merged_df) = 949
No ground truth route for suburb_bicycling walk_end, must be polygon, skipping...
No ground truth route for suburb_city_driving_weekend walk_start, must be polygon, skipping...
Processing travel leg android, ucb-sdb-android-1, accuracy_control, suburb_city_driving_weekend, 

In [ ]:
r2q_map = {"power_control": 0, "HAMFDC": 1, "MAHFDC": 2, "HAHFDC": 3, "MAMFDC": 4, "accuracy_control": 5}
q2r_map = {0: "power", 1: "HAMFDC", 2: "MAHFDC", 3: "HAHFDC", 4: "MAMFDC", 5: "accuracy"}

In [ ]:
st_errors_df["quality"] = st_errors_df.role.apply(lambda r: r2q_map[r])
st_errors_df["label"] = st_errors_df.role.apply(lambda r: r.replace('_control', ''))
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]

In [ ]:
st_errors_df.head()

In [ ]:
st_errors_df.query('timeline == "train_bus_ebike_mtv_ucb" & run == 0 & phone_os == "android" & quality == 0').error.head()

## Reference dataset choice

In [ ]:
def get_spatio_algo(pv):
    spatial_algo_list = []
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                run_errors = []
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    trip_errors = []
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        spec_name = tr["trip_id_base"]+"/"+sr["trip_id_base"]+"_"+str(r_idx)
                        if spec_name not in ref_tree[pv.spec_details.CURR_SPEC_ID]:
                            print("No reference dataset for %s, skipping" % spec_name)
                            continue
                        e = ref_tree[pv.spec_details.CURR_SPEC_ID][spec_name]
                        curr_ref_algo = {"algo": e["reference_algo"],
                             "phone_os": phone_os,
                             "phone_label": phone_label,
                             "role": r["eval_role_base"],
                             "timeline": pv.spec_details.CURR_SPEC_ID, 
                              "run": r_idx,
                              "trip_id": tr["trip_id_base"],
                              "section_id": sr["trip_id_base"]}
                        spatial_algo_list.append(curr_ref_algo)
    return spatial_algo_list

In [ ]:
spatial_algo_list = []
spatial_algo_list.extend(get_spatio_algo(pv_la))
spatial_algo_list.extend(get_spatio_algo(pv_sj))
spatial_algo_list.extend(get_spatio_algo(pv_ucb))
spatial_algo_df = pd.DataFrame(spatial_algo_list)

In [ ]:
ct_set = set(spatial_algo_df[spatial_algo_df.algo == "ct"].section_id.unique())
tf_set = set(spatial_algo_df[spatial_algo_df.algo == "tf"].section_id.unique())
print("CT always = %s,\nTF always = %s,\n BOTH = %s" % (ct_set.difference(tf_set), tf_set.difference(ct_set), tf_set.intersection(ct_set)))

## Overall stats

In [ ]:
ifig, ax_array = plt.subplots(nrows=1,ncols=2,figsize=(8,2), sharey=True)

st_errors_df.query("phone_os == 'android' & quality > 0").boxplot(ax = ax_array[0], column=["error"], by=["quality"], showfliers=False)
ax_array[0].set_title('android')
st_errors_df.query("phone_os == 'ios' & quality > 0").boxplot(ax = ax_array[1], column=["error"], by=["quality"], showfliers=False)
ax_array[1].set_title("ios")

for i, ax in enumerate(ax_array):
    # print([t.get_text() for t in ax.get_xticklabels()])
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

ax_array[0].set_ylabel("Spatio-temporal error (meters)")
# ax_array[1][0].set_ylabel("Spatial error (meters)")
ifig.suptitle("Spatio-temporal trajectory error v/s quality (excluding outliers)", y = 1.1)
# ifig.tight_layout()

In [ ]:
ifig, ax_array = plt.subplots(nrows=1,ncols=2,figsize=(8,2), sharey=True)

st_errors_df.query("phone_os == 'android' & quality > 0").boxplot(ax = ax_array[0], column=["error"], by=["quality"])
ax_array[0].set_title('android')
st_errors_df.query("phone_os == 'ios' & quality > 0").boxplot(ax = ax_array[1], column=["error"], by=["quality"])
ax_array[1].set_title("ios")

for i, ax in enumerate(ax_array):
    # print([t.get_text() for t in ax.get_xticklabels()])
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

ax_array[0].set_ylabel("Spatial-temporal error")
# ax_array[1][0].set_ylabel("Spatial error (meters)")
ifig.suptitle("Spatio-temporal trajectory error v/s quality", y = 1.1)
# ifig.tight_layout()

### Split out results by timeline

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=3,figsize=(12,6), sharex=False, sharey=False)
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]
for i, tl in enumerate(timeline_list):
    st_errors_df.query("timeline == @tl & phone_os == 'android' & quality > 0").boxplot(ax = ax_array[0][i], column=["error"], by=["quality"])
    ax_array[0][i].set_title(tl)
    st_errors_df.query("timeline == @tl & phone_os == 'ios' & quality > 0").boxplot(ax = ax_array[1][i], column=["error"], by=["quality"])
    ax_array[1][i].set_title("")

for i, ax in enumerate(ax_array[0]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

for i, ax in enumerate(ax_array[1]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

ax_array[0][0].set_ylabel("Spatio-temporal error (android)")
ax_array[1][0].set_ylabel("Spatio-temporal error (ios)")
ifig.suptitle("Spatio-temporal trajectory error v/s quality")
# ifig.tight_layout()

### Split out results by section for the most complex timeline (train_bus_ebike_mtv_ucb)

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=4,figsize=(25,10), sharex=True, sharey=True)
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    for q in range(1,5):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][q-1], column=["error"], by=["section_id"])
        ax_array[2*i][q-1].tick_params(axis="x", labelrotation=45)
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][q-1], column=["error"], by=["section_id"])
#        ax_array[i][].set_title("")

def make_acronym(s):
    ssl = s.split("_")
    # print("After splitting %s, we get %s" % (s, ssl))
    if len(ssl) == 0 or len(ssl[0]) == 0:
        return ""
    else:
        return "".join([ss[0] for ss in ssl])

for q in range(1,5):
    ax_array[0][q-1].set_title(q2r_map[q])
    curr_ticks = [t.get_text() for t in ax_array[1][q-1].get_xticklabels()]
    new_ticks = [make_acronym(t) for t in curr_ticks]
    ax_array[1][q-1].set_xticklabels(new_ticks)
    
print(list(zip(curr_ticks, new_ticks)))
# fig.text(0,0,"%s"% list(zip(curr_ticks, new_ticks)))

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = st_errors_df.query("timeline == @tl").section_id.unique()
    ifig, ax_array = plt.subplots(nrows=2,ncols=len(unique_sections),figsize=(40,10), sharex=True, sharey=False)
    for sid, s_name in enumerate(unique_sections):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][sid], column=["error"], by=["quality"])
        ax_array[2*i][sid].set_title(s_name)
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][sid], column=["error"], by=["quality"])
        ax_array[2*i+1][sid].set_title("")
#        ax_array[i][].set_title("")


### Focus only on sections where the max error is > 1000 meters

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = pd.Series(st_errors_df.query("timeline == @tl").section_id.unique())
    sections_with_outliers_mask = unique_sections.apply(lambda s_name: st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 1000)
    sections_with_outliers = unique_sections[sections_with_outliers_mask]   
    ifig, ax_array = plt.subplots(nrows=2,ncols=len(sections_with_outliers),figsize=(17,4), sharex=True, sharey=False)
    for sid, s_name in enumerate(sections_with_outliers):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][sid], column=["error"], by=["quality"])
        ax_array[2*i][sid].set_title(s_name)
        ax_array[2*i][sid].set_xlabel("")
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][sid], column=["error"], by=["quality"])
        ax_array[2*i+1][sid].set_title("")
        print([t.get_text() for t in ax_array[2*i+1][sid].get_xticklabels()])
        ax_array[2*i+1][sid].set_xticklabels([q2r_map[int(t.get_text())] for t in ax_array[2*i+1][sid].get_xticklabels() if len(t.get_text()) > 0])
        ax_array[2*i+1][sid].set_xlabel("")
    ifig.suptitle("")

### Validation of outliers

#### (express bus iOS, MAHFDC)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus' & error > 500").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus'").boxplot(column="error", by="run")

In [ ]:
import folium

In [ ]:
importlib.reload(emr)

In [ ]:
gt_leg_gj = ezgj.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/express_bus_1"]["reference_df"], color="green")
curr_map = ezgj.get_map_for_geojson(gt_leg_gj, name="ground_truth")
ezgj.get_fg_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/express_bus_1"]["reference_df"], name="gt_points", color="green").add_to(curr_map)

name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))
error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus' & run == 1"))
gt_16k = lambda lr: lr["error"] == error_df.error.max()
folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color="red"), name="sensed_values").add_to(curr_map)
ezgj.get_fg_for_loc_df(error_df, name="sensed_points", color="red", popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
gt_section = pv_ucb.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][1]["evaluation_trip_ranges"][2]["evaluation_section_ranges"][4]
gt_leg = sd_ucb.get_ground_truth_for_leg("berkeley_to_mtv_SF_express_bus", "express_bus", gt_section["start_ts"], gt_section["end_ts"]); print(gt_leg["id"])
curr_map = ezgj.get_map_for_geojson(sd_ucb.get_geojson_for_leg(gt_leg), name="ground_truth")
ezgj.get_fg_for_loc_df(emd.linestring_to_geo_df(eisd.SpecDetails.get_shapes_for_leg(gt_leg)["route"]),
                       name="gt_points", color="green").add_to(curr_map)

name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))

colors = ["red", "yellow", "blue"]
for run in range(3):
    error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus' & run == @run"))
    gt_16k = lambda lr: lr["error"] == error_df.error.max()
    print("max error for run %d is %s" % (run, error_df.error.max()))
    folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color=colors[run]), name="sensed_values").add_to(curr_map)
    ezgj.get_fg_for_loc_df(error_df, name="sensed_points", color=colors[run], popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)

folium.LayerControl().add_to(curr_map)
curr_map

#### (commuter rail aboveground android, HAMFDC)

Runs along El Camino instead of the Caltrain tracks for a while. Not sure if this is even an outlier.

In [ ]:
st_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'commuter_rail_aboveground' & error > 500").run.unique()

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-3"]["evaluation_ranges"][0]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
train_leg = to_trip["evaluation_section_ranges"][2]; print(train_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], train_leg["trip_id_base"], train_leg["start_ts"], train_leg["end_ts"]); gt_leg["id"]

In [ ]:
st_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'commuter_rail_aboveground' & error > 500").boxplot(column="error", by="run")

In [ ]:
gt_leg_gj = ezgj.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/commuter_rail_aboveground_0"]["reference_df"], color="green")
curr_map = ezgj.get_map_for_geojson(gt_leg_gj, name="ground_truth")
ezgj.get_fg_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/commuter_rail_aboveground_0"]["reference_df"], name="gt_points", color="green").add_to(curr_map)

name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))
error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'commuter_rail_aboveground' & run == 0"))
gt_16k = lambda lr: lr["error"] == error_df.error.max()
folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color="red"), name="sensed_values").add_to(curr_map)
ezgj.get_fg_for_loc_df(error_df, name="sensed_points", color="red", popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
unique_sections = pd.Series(st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb'").section_id.unique())
sections_with_outliers_mask = unique_sections.apply(lambda s_name: st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 100)
unique_sections[sections_with_outliers_mask]

#### (walk_to_bus android, HAMFDC, HAHFDC)

In the spatial-only accuracy, run 0 was the worst, with a zig-zag out to San Francisco. But the error magnitude was only ~ 3k since it wasn't that far from BART. Now that we account for temporal differences, the error is much larger (9k) but it is only a zig zag out to Ashby. I am guessing that the other error got stripped out because there was no matching timestamp. Not going to worry about that for now...

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500")

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][1]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
wb_leg = to_trip["evaluation_section_ranges"][6]; print(wb_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], wb_leg["trip_id_base"], wb_leg["start_ts"], wb_leg["end_ts"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
gt_leg_gj = ezgj.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/walk_to_bus_0"]["reference_df"], color="green")
curr_map = ezgj.get_map_for_geojson(gt_leg_gj, name="ground_truth")
ezgj.get_fg_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/walk_to_bus_0"]["reference_df"], name="gt_points", color="green").add_to(curr_map)

name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))
error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'android' & quality == 3 & section_id == 'walk_to_bus' & run == 2").sort_index(axis="index"))
gt_16k = lambda lr: lr["error"] == error_df.error.max()
folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color="red"), name="sensed_values").add_to(curr_map)
ezgj.get_fg_for_loc_df(error_df, name="sensed_points", color="red", popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)
folium.LayerControl().add_to(curr_map)
curr_map

#### (light_rail_below_above_ground, android, accuracy_control)

In this case, the spatial error was bad, but the temporal error is not that terrible, mainly because all the 

In [ ]:
st_errors_df.query("phone_os == 'android' & quality == 4 & section_id == 'light_rail_below_above_ground' & error > 100").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 4) & section_id == 'light_rail_below_above_ground'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][2]; ucb_and_back["trip_id"]
back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
lt_leg = back_trip["evaluation_section_ranges"][7]; print(lt_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(back_trip["trip_id_base"], lt_leg["trip_id_base"], lt_leg["start_ts"], lt_leg["end_ts"]); gt_leg["id"]

In [ ]:
name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))
curr_map = folium.Map()

colors = ["red", "yellow", "blue"]
for run in range(3):
    gt_loc_df = ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/light_rail_below_above_ground_%d" % run]["reference_df"]
    gt_leg = ezgj.get_geojson_for_loc_df(gt_loc_df, color=colors[i])
    gt_feature = folium.GeoJson(gt_leg, name="ground_truth_%d" % run).add_to(curr_map)
    ezgj.get_fg_for_loc_df(gt_loc_df,
                       name="gt_points_%d" % run, color="light_%s" % colors[i]).add_to(curr_map)
    error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'android' & quality == 4 & section_id == 'light_rail_below_above_ground' & run == @run"))
    gt_16k = lambda lr: lr["error"] == error_df.error.max()
    print("max error for run %d is %s" % (run, error_df.error.max()))
    folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color=colors[run]), name="sensed_values_%d" % run).add_to(curr_map)
    ezgj.get_fg_for_loc_df(error_df, name="sensed_points_%d" % run, color=colors[run], popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)

curr_map.fit_bounds(gt_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (subway, android, HAMFDC)

This is the poster child for temporal accuracy tracking

In [ ]:
gt_leg_gj = ezgj.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/subway_underground_0"]["reference_df"], color="green")
curr_map = ezgj.get_map_for_geojson(gt_leg_gj, name="ground_truth")
ezgj.get_fg_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/subway_underground_0"]["reference_df"], name="gt_points", color="green").add_to(curr_map)

name_err_time = lambda lr: "%d: %d, %s, %s" % (lr["index"], lr["df_idx"], lr["error"], sd_ucb.fmt(lr["ts"], "MM-DD HH:mm:ss"))
error_df = emd.to_loc_df(st_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'subway_underground' & run == 0").sort_index(axis="index"))
maxes = [error_df.error.max(), error_df[error_df.index > 18].error.max()]
gt_16k = lambda lr: lr["error"] in maxes
folium.GeoJson(ezgj.get_geojson_for_loc_df(error_df, color="red"), name="sensed_values").add_to(curr_map)
ezgj.get_fg_for_loc_df(error_df, name="sensed_points", color="red", popupfn=name_err_time, stickyfn=gt_16k).add_to(curr_map)
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
pd.concat([
    error_df.iloc[530:540],
    error_df.iloc[675:685]])

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 4) & section_id == 'subway_underground'").boxplot(column="error", by="run")

In [ ]:
import geojson as gj

def display_gt_and_reference(entry, loc_df_label, with_points=False):
    curr_map = folium.Map()
    # print("Using ground truth %s" % entry["ground_truth"]["leg"]["id"])
    gt_leg_gj = ezgj.get_geojson_for_linestring(entry["ground_truth"]["linestring"], color="green")
    gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
    curr_map.add_child(gt_leg_gj_feature)
    if with_points:
        gt_leg_gj_points = ezgj.get_point_markers(gt_leg_gj, name="ground_truth_points", color="green")
        curr_map.add_child(gt_leg_gj_points)
    
    sensed_location_df = entry[loc_df_label]
    # print("Adding section for %s with length %s" % (loc_df_label, len(sensed_location_df)))
    if len(sensed_location_df) > 0:
        sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(sensed_location_df.longitude, sensed_location_df.latitude))),
                                   properties={"style": {"color": "red"}, "ts": list(sensed_location_df.ts)})
        sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="reference_trajectory")
        print(sensed_leg_gj_feature)
        curr_map.add_child(sensed_leg_gj_feature)
        if with_points:
            sensed_leg_gj_points = ezgj.get_point_markers(sensed_section_gj, name="reference_points", color="red", tz="America/Los_Angeles")
            curr_map.add_child(sensed_leg_gj_points)
    
    curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
    folium.LayerControl().add_to(curr_map)
    return curr_map

def add_section_and_points(curr_map, eval_section, loc_df_label, layer_name, disp_color, with_points=False):
    sensed_location_df = eval_section[loc_df_label]
    print("Adding section for %s with length %s" % (layer_name, len(sensed_location_df)))
    sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(sensed_location_df.longitude, sensed_location_df.latitude))),
                                   properties={"style": {"color": disp_color}, "ts": list(sensed_location_df.ts)})
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values (%s)" % layer_name)
    curr_map.add_child(sensed_leg_gj_feature)
    if with_points:
        sensed_leg_gj_points = ezgj.get_point_markers(sensed_section_gj, name="sensed_points (%s)" % layer_name,
                                                      color=disp_color, tz="America/Los_Angeles")
        curr_map.add_child(sensed_leg_gj_points)

def display_gt_and_controls(entry, loc_df_label, with_points=False):
    curr_map = folium.Map()
    print("Using ground truth %s" % entry["ground_truth"]["leg"]["id"])
    gt_leg_gj = eisd.SpecDetails.get_geojson_for_leg(entry["ground_truth"]["leg"])
    gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
    curr_map.add_child(gt_leg_gj_feature)
    if with_points:
        gt_leg_gj_points = ezgj.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
        curr_map.add_child(gt_leg_gj_points)
    
    add_section_and_points(curr_map, entry["temporal_control"]["android"], loc_df_label, "android", "orange", with_points)
    add_section_and_points(curr_map, entry["temporal_control"]["ios"], loc_df_label, "ios", "purple", with_points)
    
    curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
    folium.LayerControl().add_to(curr_map)
    return curr_map


In [ ]:
##### Considering entry suburb_bicycling suburb_bicycling 0
##### max_gap for tf = 0.03349313290251375 > ct = 0.0008587982795516346 and density 0.7205317565438214 < 0.822728751810466, returning ct len = 958 not tf len = 839

sel_name = "suburb_city_driving_weekend/suburb_city_driving_weekend_3"
e = ref_tree["unimodal_trip_car_bike_mtv_la"][sel_name]
print(e["ground_truth"].keys())
display_gt_and_reference(e, "reference_df", with_points=True)

In [ ]:
ezgj.get_map_for_geojson(eisd.SpecDetails.get_geojson_for_leg(e["ground_truth"]["leg"]))